In [45]:
import numpy as np
import pandas as pd
from datetime import datetime, date

Preprocess

In [47]:
total = pd.read_csv("https://raw.githubusercontent.com/lavibula/ML20222.PredictionBitcoin/main/working/Data_working.csv")
#cac gia tri row > 4658 duoc xoa -> NULL
total = total.loc[:4657,:] 
#dien cac gia tri con thieu
total = total.fillna(method='ffill')

for index in total.index:
  total.loc[index, "Date"] = datetime.strptime(str(total.loc[index, "Date"])[:10], '%d/%m/%Y').date()
print(total.head())


         Date  BTC_close  BTC_open  BTC_high  BTC_low  BTC_volume   
0  2023-04-17    29900.0   30310.8   30312.2  29786.1       52.41  \
1  2023-04-16    30310.3   30299.2   30545.3  30134.6       34.48   
2  2023-04-15    30299.6   30472.6   30586.5  30208.8       31.71   
3  2023-04-14    30472.5   30387.4   30964.9  30026.0       98.38   
4  2023-04-13    30387.4   29892.4   30524.1  29864.5       65.87   

   Active_Addr_Cnt    Difficulty  Mean_Block_Size(in_bytes)  Sum_Block_Weight   
0              NaN           NaN                        NaN               NaN  \
1         840992.0  4.788780e+13                1866594.290       495223185.0   
2        1045660.0  4.788780e+13                1839875.354       631025193.0   
3        1016042.0  4.788780e+13                1759535.436       559166432.0   
4        1009669.0  4.788780e+13                1812112.549       567094231.0   

   ...      ETH     LTC      DOGE      XRP     GOLD  SILVER  COOPER   S&P500   
0  ...  2093.40   

In [48]:
#ETH ra doi ngay 30/7/2015, random 1 date sau do
#Biểu đồ giá đóng cửa Bitcoin (BTC) cho thấy một mô hình tương đối bằng phẳng trong dữ liệu từ tháng 9 năm 2014 đến tháng 7 năm 2016, 
#sau đó là đợt tăng giá mạnh từ tháng 9 năm 2017 đến tháng 1 năm 2018
Start_day = date(2018, 10, 1)
Test_day = date(2023,4,17)
# train, test
total = total[total["Date"] >= Start_day].reset_index(drop = True)
train_dataset = total[total["Date"] < Test_day].reset_index(drop = True)
test_dataset = total[total["Date"] >= Test_day].reset_index(drop = True)
print(test_dataset.head())
print(test_dataset.tail())

         Date  BTC_close  BTC_open  BTC_high  BTC_low  BTC_volume   
0  2023-04-17    29900.0   30310.8   30312.2  29786.1       52.41  \

   Active_Addr_Cnt  Difficulty  Mean_Block_Size(in_bytes)  Sum_Block_Weight   
0              NaN         NaN                        NaN               NaN  \

   ...     ETH    LTC      DOGE      XRP     GOLD  SILVER  COOPER  S&P500   
0  ...  2093.4  99.13  0.092952  0.51342  2022.45  25.595  8999.0     NaN  \

   DJI     JP225  
0  NaN  28514.78  

[1 rows x 24 columns]
         Date  BTC_close  BTC_open  BTC_high  BTC_low  BTC_volume   
0  2023-04-17    29900.0   30310.8   30312.2  29786.1       52.41  \

   Active_Addr_Cnt  Difficulty  Mean_Block_Size(in_bytes)  Sum_Block_Weight   
0              NaN         NaN                        NaN               NaN  \

   ...     ETH    LTC      DOGE      XRP     GOLD  SILVER  COOPER  S&P500   
0  ...  2093.4  99.13  0.092952  0.51342  2022.45  25.595  8999.0     NaN  \

   DJI     JP225  
0  NaN  28514.

In [49]:
X_train = train_dataset.drop(["Date"], axis=1)[:-1]
Y_train = train_dataset["BTC_close"][1:]

X_test = test_dataset.drop(["Date"], axis=1)[:-1]
Y_test = test_dataset["BTC_close"][1:]

Model

In [50]:
from sklearn import tree
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import KNNImputer

In [44]:
# Tìm các cột có giá trị NaN
null_cols = total.columns[total.isnull().any()].tolist()

# Tạo đối tượng KNNImputer với k = 3
imputer = KNNImputer(n_neighbors=3)

# Điền giá trị NaN bằng giá trị gần nhất sử dụng KNN
total[null_cols] = imputer.fit_transform(total[null_cols])

ValueError: at least one array or dtype is required

In [43]:
simpleTree = RandomForestRegressor(max_depth=15, n_estimators=500)
#Xây dựng cây hồi quy với độ sâu cây là 10 (Random forest), 500 cây
simpleTree.fit(X_train, Y_train)

ValueError: Input X contains NaN.
RandomForestRegressor does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:

# Importance of variables
features = X_train.columns
importances = simpleTree.feature_importances_
indices = np.argsort(importances)
plt.figure(figsize=(14,14))
plt.barh(range(len(indices)), importances[indices])
plt.yticks(range(len(indices)), features[indices])
plt.title("Variables Ranks in Period 2")
plt.show()

In [ ]:
# Importance of variables
features = X_train.columns
importances = simpleTree.feature_importances_
indices = np.argsort(importances)
plt.figure(figsize=(14,14))
plt.barh(range(len(indices)), importances[indices])
plt.yticks(range(len(indices)), features[indices])
plt.title("Variables Ranks in Period 2")
plt.show()

# Convert X_train back to a DataFrame and create feature importances DataFrame
X_train_df = pd.DataFrame(X_train, columns=X.columns)
feature_importances = pd.DataFrame({'feature': X_train_df.columns, 'importance': importances})
feature_importances = feature_importances.sort_values('importance', ascending=False).set_index('feature')

print(feature_importances)


In [ ]:
# Importance of variables
features = X_train.columns
importances = simpleTree.feature_importances_
indices = np.argsort(importances)
plt.figure(figsize=(14,12))
plt.barh(range(len(indices)), importances[indices])
plt.yticks(range(len(indices)), features[indices])
plt.title("Variables Ranks in Period 2")
plt.show()

In [ ]:
#so sanh cac gia tri du doan va quan sat
Y_pred = simpleTree.predict(X_test)
#hien thi ket qua du doan
fig, ax = plt.subplots(1, 1, figsize=(14, 5))
ax.plot(Y_test, color = 'red', label="Bitcoin Price")
ax.plot(Y_pred, color = 'green', label="Predicted Bitcoin Price", linestyle="dashed")
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, loc: "{:,}".format(int(x))))  # .3f
plt.title("Random Forest Regression for Period 2")
plt.legend()
plt.show()

In [ ]:
#tinh toan loi du doan
#RMSE
from sklearn.metrics import mean_squared_error
print("RMSE:", np.sqrt(mean_squared_error(Y_test, Y_pred)))
print()

#MAPE
MAPE = np.average(np.abs((Y_test.values - Y_pred) / Y_test.values))
print("MAPE:", MAPE)
print()

#DA
Y_test_DA = np.array(Y_test)
Y_pred_0 = Y_pred[1:] - Y_test_DA[:-1]
Y_test_0 = Y_test[1:] - Y_test_DA[:-1]
DA = (Y_pred_0<0) == (Y_test_0<0)
print("DA accuracy:", f"{round(np.mean(DA)*100, 2)}%")